# 强化学习 Atari 游戏 Demo

本笔记本演示如何使用 DQN 算法训练智能体玩 Atari 游戏 Assault-v5。

## 目录
1. [环境设置](#环境设置)
2. [数据预处理](#数据预处理)
3. [模型架构](#模型架构)
4. [训练过程](#训练过程)
5. [结果可视化](#结果可视化)
6. [模型评估](#模型评估)

In [ ]:
# 导入必要的库
import sys
import os
sys.path.append('../src')

import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from IPython.display import HTML, display
from tqdm.notebook import tqdm

# 导入自定义模块
from model import DQN, DuelingDQN
from agent import DQNAgent
from utils import make_env, plot_rewards, record_video

# 设置matplotlib中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("环境设置完成！")

## 环境设置

首先创建游戏环境并查看其基本信息。

In [ ]:
# 创建环境
env_name = 'ALE/Assault-v5'
env = make_env(env_name)

print(f"环境名称: {env_name}")
print(f"观测空间: {env.observation_space}")
print(f"动作空间: {env.action_space}")
print(f"动作数量: {env.action_space.n}")

# 重置环境并显示初始状态
state, _ = env.reset()
print(f"状态形状: {state.shape}")
print(f"状态数据类型: {state.dtype}")
print(f"状态值范围: [{state.min()}, {state.max()}]")

## 数据预处理

查看预处理后的游戏画面。

In [ ]:
# 显示预处理后的状态
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i in range(4):
    axes[i].imshow(state[i], cmap='gray')
    axes[i].set_title(f'帧 {i+1}')
    axes[i].axis('off')

plt.suptitle('预处理后的游戏状态（4帧堆叠）')
plt.tight_layout()
plt.show()

## 模型架构

创建并查看 DQN 模型的架构。

In [ ]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

# 创建模型
input_shape = env.observation_space.shape
n_actions = env.action_space.n

# 创建标准DQN模型
dqn_model = DQN(input_shape, n_actions)
print("标准DQN模型:")
print(dqn_model)

# 创建Dueling DQN模型
dueling_model = DuelingDQN(input_shape, n_actions)
print("\nDueling DQN模型:")
print(dueling_model)

# 计算模型参数数量
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n标准DQN参数数量: {count_parameters(dqn_model):,}")
print(f"Dueling DQN参数数量: {count_parameters(dueling_model):,}")

## 训练过程

演示如何训练 DQN 智能体（简化版本，用于演示）。

In [ ]:
# 创建智能体
model = DQN(input_shape, n_actions)
target_model = DQN(input_shape, n_actions)

agent = DQNAgent(
    model=model,
    target_model=target_model,
    env=env,
    device=device,
    buffer_size=10000,  # 较小的缓冲区用于演示
    batch_size=32,
    gamma=0.99,
    lr=1e-4,
    epsilon_start=1.0,
    epsilon_final=0.1,
    epsilon_decay=1000,
    target_update=100
)

print("智能体创建完成！")

In [ ]:
# 简化的训练循环（仅用于演示）
def train_demo(agent, env, n_episodes=50):
    rewards = []
    losses = []
    
    for episode in tqdm(range(n_episodes), desc="训练中"):
        state, _ = env.reset()
        episode_reward = 0
        episode_loss = 0
        steps = 0
        
        done = False
        truncated = False
        
        while not (done or truncated) and steps < 1000:  # 限制步数
            action = agent.select_action(state)
            next_state, reward, done, truncated, _ = env.step(action)
            
            agent.memory.push(state, action, reward, next_state, done)
            
            loss = agent.update_model()
            if loss is not None:
                episode_loss += loss
            
            if agent.steps_done % agent.target_update == 0:
                agent.update_target_model()
            
            state = next_state
            episode_reward += reward
            steps += 1
        
        rewards.append(episode_reward)
        losses.append(episode_loss / steps if steps > 0 else 0)
        
        if episode % 10 == 0:
            avg_reward = np.mean(rewards[-10:])
            print(f"Episode {episode}: Avg Reward = {avg_reward:.2f}")
    
    return rewards, losses

# 运行演示训练
print("开始演示训练...（这可能需要几分钟）")
rewards, losses = train_demo(agent, env, n_episodes=50)

## 结果可视化

可视化训练过程中的奖励和损失变化。

In [ ]:
# 绘制训练结果
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 奖励曲线
ax1.plot(rewards, alpha=0.7, label='回合奖励')
# 计算移动平均
window_size = 10
if len(rewards) >= window_size:
    moving_avg = [np.mean(rewards[i:i+window_size]) for i in range(len(rewards)-window_size+1)]
    ax1.plot(range(window_size-1, len(rewards)), moving_avg, 'r-', linewidth=2, label=f'{window_size}回合移动平均')

ax1.set_xlabel('回合')
ax1.set_ylabel('奖励')
ax1.set_title('训练奖励曲线')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 损失曲线
ax2.plot(losses, 'g-', alpha=0.7)
ax2.set_xlabel('回合')
ax2.set_ylabel('平均损失')
ax2.set_title('训练损失曲线')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印统计信息
print(f"平均奖励: {np.mean(rewards):.2f}")
print(f"最高奖励: {np.max(rewards):.2f}")
print(f"最低奖励: {np.min(rewards):.2f}")
print(f"奖励标准差: {np.std(rewards):.2f}")

## 模型评估

评估训练后的智能体性能。

In [ ]:
# 评估智能体
def evaluate_agent(agent, env, n_episodes=5):
    eval_rewards = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        episode_reward = 0
        steps = 0
        
        done = False
        truncated = False
        
        while not (done or truncated) and steps < 2000:
            action = agent.select_action(state, evaluate=True)  # 评估模式
            next_state, reward, done, truncated, _ = env.step(action)
            
            state = next_state
            episode_reward += reward
            steps += 1
        
        eval_rewards.append(episode_reward)
        print(f"评估回合 {episode+1}: 奖励 = {episode_reward:.2f}, 步数 = {steps}")
    
    return eval_rewards

# 运行评估
print("开始评估智能体...")
eval_rewards = evaluate_agent(agent, env, n_episodes=5)

print(f"\n评估结果:")
print(f"平均奖励: {np.mean(eval_rewards):.2f} ± {np.std(eval_rewards):.2f}")
print(f"最高奖励: {np.max(eval_rewards):.2f}")
print(f"最低奖励: {np.min(eval_rewards):.2f}")

## 总结

本演示展示了如何使用 DQN 算法训练智能体玩 Atari 游戏。主要步骤包括:

1. **环境预处理**: 将原始游戏画面转换为适合神经网络处理的格式
2. **模型设计**: 使用卷积神经网络提取视觉特征
3. **经验回放**: 存储和重用历史经验以提高学习效率
4. **目标网络**: 使用固定的目标网络稳定训练过程
5. **探索策略**: 使用ε-贪婪策略平衡探索和利用

要获得更好的性能，建议:
- 增加训练回合数
- 使用更大的经验回放缓冲区
- 尝试不同的网络架构（如 Dueling DQN）
- 使用优先经验回放
- 调整超参数